<a href="https://colab.research.google.com/github/DipankarJDutta/WBC-Biomarker-ID-for-FASD-GDM-/blob/Optimal-Batch-Size-for-MLP-model/MLP_T_cells_AS_EtOH_GDM_z_score_batch%20size.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Loading essentials for batch size (Source: scikit-learn grid search)
from numpy import loadtxt
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

Using TensorFlow backend.


In [0]:
# Fixing random seed for reproducibility
from numpy.random import seed
seed(7)
from tensorflow import set_random_seed
set_random_seed(7)

In [0]:
#Creating model required for KerasClassifier
def create_model():
    #create model
    model = Sequential()
    model.add(Dense(12, input_dim=25, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    #compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [0]:
#load z-scored inclevel values of significantly AS genes in T-cells common to EtOH & GDM datasets
dataset = loadtxt ('z-score_T-cell.csv', delimiter = ',')

In [0]:
#split dataset into input and output variables (25 input, 1 output)
x = dataset [:, 0:25]
y = dataset [:, 25]

In [0]:
#Create model for gridsearch
model = KerasClassifier (build_fn=create_model, verbose=0)

In [7]:
#Define Grid Search parameters for optimal Batch size for input T-cell dataset with 47 samples
batch_size = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47]
param_grid = dict(batch_size = batch_size)
#Jobs are NOT run in parallel with n_jobs=-1; 10-fold cross-validation
grid = GridSearchCV(estimator=model, param_grid = param_grid, n_jobs=-1, cv=10)
grid_result = grid.fit(x, y)

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


KeyboardInterrupt: ignored

In [0]:
#Summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [0]:
#Save model and architecture to a single file
model.save("DL_T-cells_batch size optimization")